In [4]:
'''1. Remove foreign records to a 'foreign dataframe' and delete them from this df. - applicable for  address
2.Find whether all phone numbers are empty, if empty drop those rows (8708)
3.Column Phone matches the Phone number in preferred phone number
Transfer records with non-null values in Phone to a new dataframe with columns EID and phone
Find the count of records (124000)
4. Null values in Phone: (36000 records) Create a new df:Check the content in preferred phone 
and grasp the preferred phone and add it to df with columns eid and phone

'''

import sys
import pandas as pd
from fuzzywuzzy import fuzz

df_txexAddr = pd.read_csv('TxExAddr.csv')
print("original count ",len(df_txexAddr))

df_Addr = df_txexAddr.dropna(how='all', subset=['Street'])
print("Count records after dropping records with no address : ",len(df_Addr))
print(df_Addr.head(3))

df_Addr = df_Addr[['EID', 'Street']]
df_Addr['EID'] = df_Addr['EID'].str.upper()
print(len(df_Addr))
print(df_Addr.head(3))

original count  166745
Count records after dropping records with no address :  158723
        EID                   Street     City     State         Zip  \
0    evr253  1715 gosnell rd apt 104   Vienna  Virginia       22182   
1     cen95        1808 Intervail Dr   Austin        TX       78746   
2  goldenr1   5735 Diamond Point Cir  El Paso        TX  79912-4160   

         Country Primary Address Type TE_Current_Address_Updated  \
0  United Statea                 Home             9/5/2019 23:01   
1  United States                 Home            2/25/2020 11:37   
2  United States                 Home             7/2/2020 10:09   

                                        Other Street Other City  \
0                                                NaN        NaN   
1             Westlake High School\n4100 Westbank Dr     Austin   
2  Western Hills Elementary School 530 Thunderbir...    El Paso   

  Other State/Province Other Zip/Postal Code Other Country  \
0                  NaN   

C:\Users\le5752\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (5,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
df_Addr1 = pd.DataFrame()
df_Addr2 = pd.DataFrame()
df_Addr1 = df_Addr[:50000]
df_Addr2 = df_Addr[50000:100000]
df_Addr3 = df_Addr[100000:]
print(len(df_Addr1))
print(df_Addr1.tail(3))
print(len(df_Addr2))
print(df_Addr2.head(3))
print(df_Addr2.tail(3))
print(len(df_Addr3))
print(df_Addr3.head(3))

50000
            EID                  Street
50541     TWT67   750 Fort Worth Avenue
50542  FARIASF1  1313 South 10th Street
50543    HELDTN       1908 Cliff Street
50000
            EID                 Street
50544  0499FJZD  7807 Moonflower Drive
50545    RRW372        5929 Merritt Pl
50546   GIRDYGO     2041 Snyder Avenue
             EID                                 Street
104693  0926JGMY                  6019 San Paublo Court
104694    NAJOLI  2611 Bens Branch Drive Apartment 2414
104695    CVC464                      455 Kayton Avenue
58723
             EID                  Street
104696    CMH573     541 Magazine Avenue
104697  JBAG7283  2800 Kirby Dr Apt A732
104698   SMS4275      14242 Bateau Drive


In [20]:
df_street = df_Addr1
#df_addrxl = df_Addr1

In [11]:
raw_eid = df_street['EID'].dropna()
eid = raw_eid.tolist()
#print(len(eid))
# How many elements each list should have 
n = 1000
pid = {}
for i in range((len(eid) + n - 1) // n ):
    #globals()["eid" + str(i)] = eid[i * n:(i + 1) * n]
    pid[i] = eid[i * n:(i + 1) * n]

In [13]:
import cx_Oracle
connection = cx_Oracle.connect("vipapp/ZJf3__6qjRt8@exa1-scan.austin.utexas.edu:1525/viprp_pdb.austin.utexas.edu")
cursor = connection.cursor()
df_addr_db = pd.DataFrame(columns = ['EID','Address1','Address2','City','State','Zip Code'])
for i in range(50):
    cursor.execute("select UT_EID, vi_addr_line1, vi_addr_line2, vi_city, vi_state,vi_zip from vi_vips_main m, vi_vips_addresses a  where m.ALUMNI_ID = a.ALUMNI_ID AND m.ut_eid IN %s" % str(tuple(pid[i])).replace(',)',')'))
    results = cursor.fetchall()
    df1 = pd.DataFrame(results, columns = ['EID','Address1','Address2','City','State','Zip Code'])
    df_addr_db = df_addr_db.append(df1,ignore_index=True)
cursor.close()
connection.close()                   

In [14]:
print(len(df_addr_db))
df_addr_db = df_addr_db.drop_duplicates()
print(len(df_addr_db))
print(df_addr_db.head(3))

99607
98385


In [21]:
df_addr_csv = df_street
print(df_addr_csv.head(4))
print(len(df_addr_csv))
del df_street

        EID                   Street
0    EVR253  1715 gosnell rd apt 104
1     CEN95        1808 Intervail Dr
2  GOLDENR1   5735 Diamond Point Cir
3     RHM88     2912 River Pine Lane
50000


In [22]:
addr_diff = pd.DataFrame()
for i in range(len(df_addr_csv)):
#for i in (11,44,55,66,123,133,143,333,445,667,246,988):
    flag = 0
    id_csv = df_addr_csv.iloc[i,0]
    csv_addr = df_addr_csv.iloc[i,1]
    cmp_dbaddr = df_addr_db.loc[df_addr_db['EID'] == id_csv]
    #print(cmp_dbaddr)
    #print(df_addr_csv.loc[df_addr_csv['EID'] == id_csv])
    #print("---")
    
    for j in range(len(cmp_dbaddr)):
        db_addr = cmp_dbaddr.iloc[j,1]
        Token_Set_Ratio = fuzz.token_set_ratio(csv_addr,db_addr)
        if (Token_Set_Ratio > 77):
            flag = 0
            break
        else:
            flag = 1
    
    #print("flag = ", flag)
    
    if (flag == 1):
        addrlist = []
        for k in range(len(cmp_dbaddr)):
            temp = (cmp_dbaddr.loc[cmp_dbaddr['EID'] == id_csv]).iloc[:,1]
            addrlist = temp.values.tolist()
        addr_diff = addr_diff.append({'EID':id_csv, 'DB_Address':addrlist,'CSV_Address':csv_addr}, ignore_index=True)
#print("----------------------------------------------------------------")
#print(addr_diff)
#print("----------------------------------------------------------------")

In [24]:
print(addr_diff.head(11))
print(len(addr_diff))

                         CSV_Address  \
0            1715 gosnell rd apt 104   
1              2900 Manor Rd., #3362   
2             203 Burning Tree Drive   
3            2472 Worthington Street   
4           2401 Aldrich St. Apt.343   
5   12367 Spanish Trace Drive, Apt G   
6                             Unit A   
7         2312 Zirkell Lane Apt. 202   
8           5400 N Lamar Blvd. #3030   
9                3104 Lynridge Drive   
10             6003 Running Creek Ct   

                                           DB_Address       EID  
0               [2313 Sequoia Ln, 2810 Hemphill Park]    EVR253  
1   [The University of Texas at Austin, 2208 Del C...    CNG829  
2   [The University of Texas at Austin, 12704 Lowd...   EVANSPA  
3   [The University of Texas at Austin, 1103 Black...    WRF374  
4   [The University of Texas at Austin, 2401 Aldri...   JH68833  
5   [The University of Texas at Austin, 1811 Sycam...   SA44965  
6   [4127 Honeycomb Rock Circle, The University of...  SK

In [25]:
addr_diff.to_csv(r'C:\Users\le5752\Documents\DB_Python\addr1.csv', index = False)